# Kontur.fokus ultimate beneficiary scraper

Aim here is to feed API queries with tons of INN (taxpayer numbers) to Kontur fokus API, to extract all info about shareholders, CEOs and affiliated companies from it.

### Section 1: Setup

In [1]:
import requests
import re
from requests.adapters import HTTPAdapter
from urllib.parse import urlparse, parse_qs
from urllib3.util import Retry
from bs4 import BeautifulSoup,SoupStrainer
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
from datetime import datetime, date, timedelta
import logging
import http.client
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
from os import walk
import json
import numpy as np
import csv
from urllib import response
from webbrowser import get


In [2]:
with open(r"C:\Users\majda\Desktop\inn.csv") as f:
    sample = [str(s) for line in f.readlines() for s in line[:-1].split(',')]
    print(sample)

['6680000422', '6680000447', '6680001345', '6680001722', '6680002370', '6680005928', '6680007731', '6681000834', '6681005568', '6681005960', '6681007572', '6682001414', '6682004334', '6682007342', '6683000011', '6683003870', '6683005482', '6683005789', '6683007257', '6683010394', '6683010806']


In [67]:

def getData():
    output=[]
    id=[]
    est=[]
    cap=[]
    bene=[]
    shares=[]
    historical=[]
    
    inn=['6608005109']
    
    for i in inn:
        print(i)
        temp=f"https://focus-api.kontur.ru/api3/beneficialOwners?inn={i}&key=4bcd0b82f0b0ea748eb8a984e47a0a6a48426667"
        response = requests.get(temp)
        id=response.json()[0]['inn']
        
        try:
            cap=response.json()[0]['statedCapital']['sum']
            est=response.json()[0]['statedCapital']['date']
        except:
            est=("")
            cap=("")
                    
        owners = response.json()[0]['beneficialOwners']
        Howners= response.json()[0]['historicalBeneficialOwners']
        
         # ponizej for nie hula :/
        
        
        for FLowners in owners['beneficialOwnersFL']:     
            n=len(response.json()[0]['beneficialOwners']['beneficialOwnersFL'])
            m=list(range(n))
            o=print(m)
            
            if n==0:
                fio='N/A'
                shares='N/A'
                output.append({'id':i, 'bene':fio, 'shares':shares,'ownerType': 'FL'})
            else:
            
                fio=owners['beneficialOwnersFL'][m]['fio']
                # FLinn=owners['beneficialOwnersFL'][m]['innfl']
                shares=owners['beneficialOwnersFL'][m]['share']
                output.append({'id':i, 'bene':fio, 'shares':shares,'ownerType': 'FL'})

        # for ownerFGN in owners['beneficialOwnersForeign'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersForeign'])
            
        #     FGNname=owners['beneficialOwnersForeign'][n-1]['fullName']
        #     shares=owners['beneficialOwnersForeign'][n-1]['share']            
        #     output.append({'id':i, bene:FGNname, shares:shares, 'ownerType': 'FGN'})
            
        # for ownerUL in owners['beneficialOwnersUL'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersUL'])

        #     ULname=owners['beneficialOwnersUL'][0]['fullName']
        #     shares=owners['beneficialOwnersUL'][0]['share']
        #     output.append({'id':i, bene:ULname, shares:shares, 'ownerType': 'UL'})

        # for ownerOth in owners['beneficialOwnersOther'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersUL'])

        #     Oname=owners['beneficialOwnersOther'][n-1]['fullName']
        #     shares=owners['beneficialOwnersOther'][n-1]['share']            
        #     output.append({'id':i, bene:Oname, shares:shares, 'ownerType': 'Other'})
            
        ### HISTORICAL ###    
            
        # for HownerFL in Howners['beneficialOwnersFL'][n-1]: # aleee musi zaczac od 0 a nie od 1...
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersFL'])
            
        #     fio=Howners['beneficialOwnersFL'][n-1]['fio']
        #     FLinn=Howners['beneficialOwnersFL'][n-1]['innfl']
        #     shares=Howners['beneficialOwnersFL'][n-1]['share']
        #     output.append({'id':i, bene:fio, shares:shares,'ownerType': 'FL'})

        # for HownerFGN in Howners['beneficialOwnersForeign'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersForeign'])
            
        #     FGNname=Howners['beneficialOwnersForeign'][n-1]['fullName']
        #     shares=Howners['beneficialOwnersForeign'][n-1]['share']            
        #     output.append({'id':i, bene:FGNname, shares:shares, 'ownerType': 'FGN'})
            
        # for HownerUL in Howners['beneficialOwnersUL'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersUL'])

        #     ULname=Howners['beneficialOwnersUL'][0]['fullName']
        #     shares=Howners['beneficialOwnersUL'][0]['share']
        #     output.append({'id':i, bene:ULname, shares:shares, 'ownerType': 'UL'})

        # for HownerOth in Howners['beneficialOwnersOther'][n-1]:
        #     n=len(response.json()[0]['beneficialOwners']['beneficialOwnersUL'])

        #     Oname=Howners['beneficialOwnersOther'][n-1]['fullName']
        #     shares=Howners['beneficialOwnersOther'][n-1]['share']            
        #     output.append({'id':i, bene:Oname, shares:shares, 'ownerType': 'Other'})

            

    
    return output

In [68]:
data=getData()

print(len(data))
print(data) 


6608005109
[0, 1, 2, 3, 4]
5
[0, 1, 2, 3, 4]


TypeError: list indices must be integers or slices, not list

In [32]:
df=pd.DataFrame(data,columns=['id','est', 'ownerType', 'bene','historical'])
df.to_csv("APItest!")